<a href="https://colab.research.google.com/github/SWLee1212/TensorFlow-in-Practice/blob/master/NLP_Week4_gen_text_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.0.0


In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [5]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [6]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))
print(vocab)

고유 문자수 65개
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

print(char2idx)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [8]:
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [0]:
text_as_int = np.array([char2idx[c] for c in text])

In [10]:
print(text_as_int[:20])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [11]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [12]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])
  
print("{} of examples per epch".format(examples_per_epoch))  

F
i
r
s
t
11153 of examples per epch


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

print(sequences)
for item in sequences.take(1):
  print(len(item.numpy()))
  print(repr(''.join(idx2char[item.numpy()])))

<BatchDataset shapes: (101,), types: tf.int64>
101
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
print(tf.data.experimental.cardinality(sequences).numpy())
print(len(text_as_int)/101)
print(examples_per_epoch)

11043
11043.50495049505
11153


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 18 ('F')
  예상 출력: 47 ('i')
   1단계
  입력: 47 ('i')
  예상 출력: 56 ('r')
   2단계
  입력: 56 ('r')
  예상 출력: 57 ('s')
   3단계
  입력: 57 ('s')
  예상 출력: 58 ('t')
   4단계
  입력: 58 ('t')
  예상 출력: 1 (' ')


In [18]:
# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [22]:
vocab_size

65

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(input_example_batch.shape)
  print(target_example_batch.shape)
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)


(64, 100)
(64, 100)
(64, 100, 65)


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(example_batch_predictions[0].shape,sampled_indices.shape )
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices.shape)

(100, 65) (100, 1)
(100,)


In [26]:
print(example_batch_predictions[0])

tf.Tensor(
[[-0.00115136 -0.00059436 -0.00134415 ... -0.00491749  0.00322246
   0.00245086]
 [ 0.00241114  0.00694487 -0.00781566 ...  0.00059266  0.00262329
  -0.00465093]
 [ 0.00419355  0.01063012 -0.00360911 ... -0.00411889 -0.00267353
  -0.00835107]
 ...
 [-0.00250592  0.00896428 -0.00078815 ... -0.00857972  0.004391
  -0.00278685]
 [-0.00523379  0.00935444  0.00346116 ... -0.01033749 -0.00089915
  -0.00619626]
 [-0.00154993  0.0067698   0.00350821 ... -0.01002302 -0.00950833
  -0.00515196]], shape=(100, 65), dtype=float32)


In [27]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 " as you are weary of the weight,\nRest you, whiles I lament King Henry's corse.\n\nGLOUCESTER:\nStay, yo"

예측된 다음 문자: 
 "KslUcWDDVfJsjZHzfbwFzyglkkL$BAJLG& Z$rYpdlOo?O,?\nzB?SHmqjOKEOe'Y$iIF&YbhfOF3BALah3bKiwmC\nUucziFDsQup"


In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.1738243


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [31]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 33s 193ms/step - loss: 2.5839
Epoch 2/50
172/172 [==============================] - 31s 180ms/step - loss: 1.8893
Epoch 3/50
172/172 [==============================] - 31s 179ms/step - loss: 1.6417
Epoch 4/50
172/172 [==============================] - 31s 179ms/step - loss: 1.5069
Epoch 5/50
172/172 [==============================] - 31s 179ms/step - loss: 1.4258
Epoch 6/50
172/172 [==============================] - 31s 179ms/step - loss: 1.3679
Epoch 7/50
172/172 [==============================] - 31s 179ms/step - loss: 1.3218
Epoch 8/50
172/172 [==============================] - 31s 179ms/step - loss: 1.2831
Epoch 9/50
172/172 [==============================] - 31s 179ms/step - loss: 1.2457
Epoch 10/50
172/172 [==============================] - 31s 179ms/step - loss: 1.2104
Epoch 11/50
172/172 [==============================] - 31s 179ms/step - loss: 1.1744
Epoch 12/50
172/172 [==============================] - 31s 179ms/step - lo

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)  
  
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

예측된 다음 문자: 
 'HIchard Gath best deserved df rll.my hons.\n\nut is your grace,dead  my Lord of Sometset?\nONORFOLK:\nHo'


In [33]:
print("입력 라벨 문자: \n", repr("".join(idx2char[target_example_batch[0].numpy()])))

입력 라벨 문자: 
 'Richard hath best deserved of all my sons.\nBut is your grace dead, my Lord of Somerset?\n\nNORFOLK:\nSu'


In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
# acc=history.history['acc']
# val_acc=history.history['val_acc']
loss=history.history['loss']
# val_loss=history.history['val_loss']

epochs=range(len(loss)) # Get number of epochs

In [38]:
fig = go.Figure(
    data = [go.Scatter(x=list(epochs), y=loss)]
)
fig.show()